In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
import time
warnings.filterwarnings('ignore')
import json
from sklearn.model_selection import RandomizedSearchCV

In [0]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
data = pickle.load(open("/content/drive/My Drive/New_data/Kowshik_Srijan_NER_without_vocab/Data/tfidf_pca_result_final_500.pickle", "rb"))
X_pred = (pd.read_pickle("/content/drive/My Drive/New_data/Kowshik_Srijan_NER_without_vocab/Data/GS_tfidf_pca_result_final_500.pickle")).drop(columns = ['doc_id'])

gc.collect()

0

In [0]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
RandomForestClassifier()

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:


model = RandomForestClassifier()

params_dist = {
    "n_estimators":[100],
    "min_samples_leaf": np.arange(1,60,10),
    "max_leaf_nodes": np.arange(2,60,10)
}

start_time = time.time()

optimal_model = RandomizedSearchCV(model,params_dist,scoring = "f1",n_jobs = -1)
y_pred = optimal_model.fit(X_train,y_train).predict(X_test)

end_time = time.time()

print("Elapsed Time:", time.strftime("%H:%M:%S", time.gmtime(end_time - start_time)))

Elapsed Time: 05:03:43


In [0]:
score = f1_score(y_test,y_pred)

print("====================")
print("f1 score: {}".format(score))
print("Best Score: {}".format(optimal_model.best_score_))
print("Best Parameters: {}".format(optimal_model.best_params_))

f1 score: 0.4760565671589761
Best Score: 0.47171243866954615
Best Parameters: {'n_estimators': 100, 'min_samples_leaf': 31, 'max_leaf_nodes': 52}


In [0]:
start_time = time.time()

model = RandomForestClassifier()

model.fit(X_train, y_train)
print(model.score(X_test, y_test))

stop_time = time.time()
print("Elapsed Time:", time.strftime("%H:%M:%S", time.gmtime(stop_time - start_time)))

pred = model.predict(X_test)
print(f1_score(pred, y_test))

0.7966292247116903
Elapsed Time: 00:40:52
0.5823782308457915


In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/output files/GS_sent_500.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = model.predict(X_pred)
probability_of_predictions = model.predict_proba(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions == 1])
result['probabilities'] = probability_of_predictions[predictions == 1][:,1]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

,Sents,doc_id,probabilities
1,1869 of 1968. appeal under s. 116-a of the rep...,0.0,0.605525
2,"16 of 1967. s. v. gupte, m. m. tiwari, h. k. p...",0.0,0.638786
5,"rajasthan and k. b. mehta, for the state of ra...",0.0,0.627638
13,"we are concerned only with one of them, namely...",0.0,0.518319
14,"838 article 191 of the constitution, which is ...",0.0,0.631311
...,...,...,...
6923,"but the word ""reduction"" has no such limitatio...",48.0,0.570852
6949,"the learned counsel, however, could not substa...",48.0,0.539471
7046,13. the point now is as to what orders are to ...,48.0,0.506535
7072,"21 'and 41 -of 1952, filed in the high court o...",49.0,0.528256
